In [109]:
import pandas as pd
import numpy as np
import random
from scipy import stats
from sklearn.cluster import KMeans
from itertools import permutations
pd.__version__
import cv2
from sklearn.metrics import silhouette_score
from statistics import mean
from statistics import harmonic_mean
import math

In [110]:
#user inputs

#Read colorlith image file in BGR format
img = cv2.imread("Input_colorlith.png")

#input start and end depths
start_depth=1460 #m
end_depth=1520 #m

# do you have core data, make this value 1 if you have core data, otherwise make it 0
have_data=1
# Input the number, names and properties of homogeneous lithologies recorded in core
if(have_data==1):
    # User input for the number of the lithologies
    Sst_num = 2
    CbSst_Num = 1
    Mst_Num = 2
    # User input for the name of the lithologies
    # Sandstone names should be increasing order of grain sizes
    Sand_Name = ["Sst1","Sst2"] #Sst1=Fine sandstone; Sst2=Coarse sandstone
    Carb_Name = ["CbSst1"] #CbSst1=Carbonate-cemented sandstone
    # Mudstone should be decreasing order of grain sizes
    Mud_Name =  ["Mst1","Mst2"] #Mst1=Siltstone; Mst2=Mudstone

#The abbreviations are replaced with the original names in the final output

In [111]:
#variables calculated internally

#image parameters
row_image= int(img.shape[0])
column_image=img.shape[1]
pixel_height= ((end_depth-start_depth)/row_image)
df_RGB=pd.DataFrame(columns=['Depth','Red','Green','Blue'])
a=0
# Putting RGB from an image into the dataframe
for a in range(row_image):
    df_RGB = df_RGB.append({'Depth':start_depth+(a*pixel_height),
    'Blue':img[a][column_image-50][0],
    'Green':img[a][column_image-50][1],
    'Red':img[a][column_image-50][2]}, ignore_index=True)

#master rock type list
All_rock_types=Sand_Name+Carb_Name+Mud_Name


In [112]:
#Calculating Z scores for R,G,B values
if(df_RGB.Red.std()==0):
    df_RGB['Z_Red']=0
if(df_RGB.Red.std()!=0):
    df_RGB['Z_Red']= (df_RGB.Red-df_RGB.Red.mean())/(df_RGB.Red.std())

if(df_RGB.Green.std()==0):
    df_RGB['Z_Green']=0
if(df_RGB.Green.std()!=0):
    df_RGB['Z_Green']= (df_RGB.Green-df_RGB.Green.mean())/(df_RGB.Green.std())    

if(df_RGB.Blue.std()==0):
    df_RGB['Z_Blue']=0
if(df_RGB.Blue.std()!=0):
    df_RGB['Z_Blue']= (df_RGB.Blue-df_RGB.Blue.mean())/(df_RGB.Blue.std())

In [113]:
#clustering if user has the number of rock types
if(have_data==1):
    km_Red = KMeans(n_clusters = (Sst_num+1))
    km_Green = KMeans(n_clusters= (CbSst_Num+1))
    km_Blue =KMeans(n_clusters= (Mst_Num+1))

#clustering if the user does not have the number of rock types
if(have_data==0):
    sil_red_score=[None]*8
    sil_green_score=[None]*8
    sil_blue_score=[None]*8
    
    i=0
    for i in range (2,10):
        km_Red = KMeans(n_clusters = (i))
        y_Red=km_Red.fit_predict(df_RGB[['Z_Red']])
        sil_red_score[i-2]=silhouette_score(df_RGB[['Z_Red']],y_Red)

        km_Green = KMeans(n_clusters= (i))
        y_Green=km_Green.fit_predict(df_RGB[['Z_Green']])
        sil_green_score[i-2]=silhouette_score(df_RGB[['Z_Green']],y_Green)

        km_Blue = KMeans(n_clusters= (i))
        y_Blue=km_Blue.fit_predict(df_RGB[['Z_Blue']])
        sil_blue_score[i-2]=silhouette_score(df_RGB[['Z_Blue']],y_Blue)
    
    # Auto input of the number and name of the lithologies
    Sst_Num = np.argmax(sil_red_score)+2
    CbSst_Num = np.argmax(sil_green_score)+2
    Mst_Num = np.argmax(sil_blue_score)+2
    
    Sand_Name=[None]*Sst_Num
    Carb_Name=[None]*CbSst_Num
    Mud_Name=[None]*Mst_Num
    i=0
    for i in range (Sst_Num):
        Sand_Name[i]="Sst"+str(i+1)
    i=0
    for i in range (CbSst_Num):
        Carb_Name[i]="CbSst"+str(i+1)
    i=0
    for i in range (Mst_Num):
        Mud_Name[i]="Mst"+str(i+1)

    All_rock_types=Sand_Name+Carb_Name+Mud_Name

    km_Red = KMeans(n_clusters = (np.argmax(sil_red_score)+2))
    km_Green = KMeans(n_clusters= (np.argmax(sil_green_score)+2))
    km_Blue =KMeans(n_clusters= (np.argmax(sil_blue_score)+2))

In [114]:
#Computing local cluster IDs for R,G and B columns

#new dataframe for Red

df_Red = pd.DataFrame(columns=['Z_Red','Red'])
df_Red['Red']= df_RGB['Red']
df_Red['Z_Red']= df_RGB['Z_Red']
df_Red['Red_local_Cluster_ID']=(km_Red.fit_predict(df_RGB[['Z_Red']]))
df_RGB['Red_local_Cluster_ID'] = df_Red['Red_local_Cluster_ID']

#new dataframe for Green

df_Green = pd.DataFrame(columns=['Z_Green','Green'])
df_Green['Green']= df_RGB['Green']
df_Green['Z_Green']= df_RGB['Z_Green']
df_Green['Green_local_Cluster_ID']=(km_Green.fit_predict(df_RGB[['Z_Green']]))
df_RGB['Green_local_Cluster_ID'] = df_Green['Green_local_Cluster_ID']

#new dataframe for Blue

df_Blue = pd.DataFrame(columns=['Z_Blue','Blue'])
df_Blue['Blue']= df_RGB['Blue']
df_Blue['Z_Blue']= df_RGB['Z_Blue']
df_Blue['Blue_local_Cluster_ID']=(km_Blue.fit_predict(df_RGB[['Z_Blue']]))
df_RGB['Blue_local_Cluster_ID'] = df_Blue['Blue_local_Cluster_ID']

In [115]:
#Min-max range for each local cluster

#Characteristic properties of the Red column
df_Red_prop = pd.DataFrame(columns=['Red_min','Red_max'])
for b in range ((Sst_num+1)):
        max=0
        min=255
        for a in range(int(len(df_RGB))):
            if(df_Red["Red_local_Cluster_ID"][a] == b):
                if(df_Red["Red"][a]>max):
                        max=df_Red["Red"][a]
                if(df_Red["Red"][a]<min):
                        min=df_Red["Red"][a]
        df_Red_prop = df_Red_prop.append({'Red_min': min, 'Red_max':max}, ignore_index=True)


#Characteristic properties of the Green column
df_Green_prop = pd.DataFrame(columns=['Green_min','Green_max'])
for b in range ((CbSst_Num+1)):
        max=0
        min=255
        for a in range(int(len(df_RGB))):
            if(df_Green["Green_local_Cluster_ID"][a] == b):
                if(df_Green["Green"][a]>max):
                        max=df_Green["Green"][a]
                if(df_Green["Green"][a]<min):
                        min=df_Green["Green"][a]
        df_Green_prop = df_Green_prop.append({'Green_min': min, 'Green_max':max}, ignore_index=True)


#Characteristic properties of the Blue column
df_Blue_prop = pd.DataFrame(columns=['Blue_min','Blue_max'])
for b in range ((Mst_Num+1)):
        max=0
        min=255
        for a in range(int(len(df_RGB))):
            if(df_Blue["Blue_local_Cluster_ID"][a] == b):
                if(df_Blue["Blue"][a]>max):
                        max=df_Blue["Blue"][a]
                if(df_Blue["Blue"][a]<min):
                        min=df_Blue["Blue"][a]
        df_Blue_prop = df_Blue_prop.append({'Blue_min': min, 'Blue_max':max}, ignore_index=True)


# Sorting the individual dataframes according to the minimum values
df_Red_prop=df_Red_prop.sort_values(by='Red_min', ascending=True, ignore_index=True)
df_Green_prop=df_Green_prop.sort_values(by='Green_min', ascending=True, ignore_index=True)
df_Blue_prop=df_Blue_prop.sort_values(by='Blue_min', ascending=True,ignore_index=True)

In [116]:
#Calculating global cluster IDs
df_RGB["Red_Global_cluster_ID"]=0
df_RGB["Green_Global_cluster_ID"]=0
df_RGB["Blue_Global_cluster_ID"]=0

for a in range(int(len(df_RGB))):
    b=0
    for b in range(int(len(df_Red_prop))):
        if(df_RGB["Red"][a]>=df_Red_prop["Red_min"][b] and df_RGB["Red"][a]<=df_Red_prop["Red_max"][b]):
            df_RGB.loc[a,"Red_Global_cluster_ID"] = b
    b=0
    for b in range(int(len(df_Green_prop))):
        if(df_RGB["Green"][a]>=df_Green_prop["Green_min"][b] and df_RGB["Green"][a]<=df_Green_prop["Green_max"][b]):
            df_RGB.loc[a,"Green_Global_cluster_ID"] = b
    b=0
    for b in range(int(len(df_Blue_prop))):
        if(df_RGB["Blue"][a]>=df_Blue_prop["Blue_min"][b] and df_RGB["Blue"][a]<=df_Blue_prop["Blue_max"][b]):
            df_RGB.loc[a,"Blue_Global_cluster_ID"] = b

In [117]:
#Deducing constituent lithologies from the global cluster IDs for each row of the image

df_RGB['Lithology'] = ""

for a in range(int(len(df_RGB))):
    c=3 
    #counter for the number of non zero and member rock types at particular depth 
    #0- no rock found
    #1 - homogeneous rock
    #2 and 3 - composite rock 
    if (df_RGB["Red_Global_cluster_ID"][a]==0):
        c-=1
    if (df_RGB["Green_Global_cluster_ID"][a]==0):
        c-=1    
    if (df_RGB["Blue_Global_cluster_ID"][a]==0):
        c-=1
    if(c==0):  # checking for no rocks
               # if no rocks found, assiging the rock type of the highest color code
        if(df_RGB["Red"][a] > df_RGB["Green"][a] and df_RGB["Red"][a] > df_RGB["Blue"][a]):
            df_RGB.loc[a,"Lithology"] = "Homogeneous " + Sand_Name[0]
        if(df_RGB["Green"][a] > df_RGB["Red"][a] and df_RGB["Green"][a] > df_RGB["Blue"][a]):
            df_RGB.loc[a,"Lithology"] = "Homogeneous " + Carb_Name[0]
        if(df_RGB["Blue"][a] > df_RGB["Red"][a] and df_RGB["Blue"][a] > df_RGB["Green"][a]):
            df_RGB.loc[a,"Lithology"] = "Homogeneous " + Mud_Name[0]
    if (c==1): # checking for homogeneous rock types
        if(df_RGB["Red_Global_cluster_ID"][a]>0):
            df_RGB.loc[a,"Lithology"] = "Homogeneous " + Sand_Name[int(df_RGB["Red_Global_cluster_ID"][a])-1]
        if(df_RGB["Green_Global_cluster_ID"][a]>0):
            df_RGB.loc[a,"Lithology"] = "Homogeneous " + Carb_Name[int(df_RGB["Green_Global_cluster_ID"][a])-1]
        if(df_RGB["Blue_Global_cluster_ID"][a]>0):
            df_RGB.loc[a,"Lithology"] = "Homogeneous " + Mud_Name[int(df_RGB["Blue_Global_cluster_ID"][a])-1]
    if (c>=2): #  checking for hetrogeneous rock types
        if(df_RGB["Red_Global_cluster_ID"][a]>0 and df_RGB["Green_Global_cluster_ID"][a]>0 ):
            df_RGB.loc[a,"Lithology"] = "Composite of " + Sand_Name[int(df_RGB["Red_Global_cluster_ID"][a])-1] +" " + Carb_Name[int(df_RGB["Green_Global_cluster_ID"][a])-1]
        if(df_RGB["Blue_Global_cluster_ID"][a]>0 and df_RGB["Green_Global_cluster_ID"][a]>0 ):
            df_RGB.loc[a,"Lithology"] = "Composite of " + Mud_Name[int(df_RGB["Blue_Global_cluster_ID"][a])-1] +" " + Carb_Name[int(df_RGB["Green_Global_cluster_ID"][a])-1]
        if(df_RGB["Red_Global_cluster_ID"][a]>0 and df_RGB["Blue_Global_cluster_ID"][a]>0 ):
            df_RGB.loc[a,"Lithology"] = "Composite of " + Sand_Name[int(df_RGB["Red_Global_cluster_ID"][a])-1] +" " + Mud_Name[int(df_RGB["Blue_Global_cluster_ID"][a])-1]
        if(df_RGB["Red_Global_cluster_ID"][a]>0 and df_RGB["Blue_Global_cluster_ID"][a]>0 and df_RGB["Green_Global_cluster_ID"][a]>0  ):
            df_RGB.loc[a,"Lithology"] = "Composite of " + Sand_Name[int(df_RGB["Red_Global_cluster_ID"][a])-1] +" " + Carb_Name[int(df_RGB["Green_Global_cluster_ID"][a])-1] +" " + Mud_Name[int(df_RGB["Blue_Global_cluster_ID"][a])-1]

In [118]:
#Copying only the depth and lithology columns in a separate dataframe 
df_copy= pd.DataFrame(columns=['Depth','Lithology'])
data=df_RGB[["Depth","Lithology"]]
df_copy=data.copy()

In [121]:
# Uneven averaging from input pixel height (3.5cm)) to nearest multiple of 5

df_coarse= pd.DataFrame(columns=['Depth','Lithology','LITHOLOGY'])

litho_coarse = ""

i=0
coarse_reso= (((3.5 // 5)+1)*5)/100
residual=0
row_coarse = (end_depth-start_depth)/coarse_reso 
j=0
k=0
for j in range (int(row_coarse)):
    df_coarse = df_coarse.append({'Depth':start_depth+(j*coarse_reso)},ignore_index=True)
    # df_coarse["Depth"][j]=start_depth+(j*coarse_reso)
   
for k in range (int(row_image)):
    if(residual<pixel_height):
        a=i+1
        b=k+1
        df_coarse.loc[i,"Lithology"]= df_copy.loc[k,"Lithology"]
        # df_coarse['Lithology'][i]= litho_coarse + " " + df_copy['Lithology'][k]
        residual= ((coarse_reso*(a))-(pixel_height*(b)))
        i+=1
    if(residual>pixel_height):
        # df_coarse['Lithology'][i-1]=""
        a=i-1
        b=k-1
        df_coarse.loc[a,"Lithology"]= df_copy.loc[b,"Lithology"] +" " + df_copy.loc[k,"Lithology"]
        residual=residual-pixel_height

n=0
for n in range (int(len(df_coarse))):
    if(math.isnan(df_coarse.loc[n, "Depth"])==True):
        df_coarse.loc[n, "Depth"]=start_depth+n*coarse_reso

,Depth,Lithology,LITHOLOGY
0,1460.00,,NaN
1,1460.05,Homogeneous Mst2,NaN
2,1460.10,Homogeneous Mst2 Homogeneous Mst2,NaN
3,1460.15,Homogeneous Mst2 Homogeneous Mst2,NaN
4,1460.20,Homogeneous Mst2 Homogeneous Mst2,NaN
...,...,...,...
1196,1519.80,Composite of Sst1 Mst1 Composite of Sst2 Mst1,NaN
1197,1519.85,Composite of Sst2 Mst1 Composite of Sst2 Mst1,NaN
1198,1519.90,Composite of Sst2 Mst1 Composite of Sst1 Mst1,NaN
1199,1519.95,Composite of Sst1 Mst1 Composite of Sst1 Mst1,NaN


In [122]:
#getting the nearest lithology if the first index has no value
str_start= df_coarse['Lithology'][0]
a=0
for a in range(len(df_coarse)):
    if(str_start ==""):
        b=a+1
        str_start= df_coarse.loc[b,"Lithology"]
        c+=1

if(c>0):
    x=0
    for x in range(b):
        df_coarse.loc[x,"Lithology"]= str_start


#getting the nearest lithology if the Last index has no value 
a=int(len(df_coarse)-1)
str_end= df_coarse.loc[a,"Lithology"] 
c=0
a=0
for a in range(len(df_coarse)):
    if(str_end ==""):
        b=int(len(df_coarse)-a-1)
        str_end= df_coarse.loc[b,"Lithology"]
        c+=1
    else:
        break

if(c>0):
    x=0
    for x in range(int(row_coarse)):
        b1=int(len(df_coarse)-x-1)
        str_end1= df_coarse.loc[b1,"Lithology"]
        if(str_end1 ==""):
            df_coarse.loc[b1,"Lithology"]= str_end
        else:
            break

#refining the string name and putting in a new column in the dataframe
a=0
for a in range(len(df_coarse)):
    str= df_coarse.loc[a,"Lithology"]
    str3=list(dict.fromkeys(str.split(" ")))
    for b in range(len(str3)):
        if(str3[b]=='Homogeneous' or str3[b]=='Composite' or str3[b]=='of'):
            str3[b]=""
    c=0
    for x in range(len(str3)):
        if(str3[x]==""): c+=1
    for y in range(c):
        str3.remove("")
    str3.sort()
#writing Homogeneous if only one rock and composite if two or more than two rocks
    if (len(str3)==1):
        str3.insert(0,'Homogeneous')
    else:
        str3.insert(0,'Composite of')
    df_coarse.loc[a,"LITHOLOGY"]= ' '.join(str3)

In [123]:
# output of the quantified colourlith log
df_coarse.to_csv("Output_colorlith.csv", columns=["Depth","LITHOLOGY"])

In [124]:
#The following code (till the end) computes upscaled properties of each rock type identified in the quantified colorlith log

#Upscaled abs perm is calculated along x, y and z directions
#Upscaled rel-perm is calculated along x, y and z directions under capillary and viscous limits
#Upscaled porosity and Capillary pressure are pore volume weighted

#Saturation functions are calculated for Van-Genuchten formulation

#Properties are calculated under the following assumptions:
# a. If a composite rock type has mudstone, we assume horizontal laminations to be dominant
# b. If a composite rock type has carbonate-cement, we assume a homogeneous overprinting of pre-existing lamination
# c. If a composite rock type has only sandstones, we assume oblique laminations to be dominant
# d. The proportion of all the constituents of a composite is equal
# e. The layer thickness of all the constituents of a composite is equal

In [125]:
#User input for minimum, maximum and mean porosities and absolute permeabilites (mD); residual water saturation, 
#Van-Genuchten exponent, capillary entry pressures in the sequence Sandstone, Carbonates and Mudstones
#as well as fluid viscosities 

#porosity
poro_min=[0.25,0.25,0.05,0.1,0.05]
poro_max=[0.35,0.35,0.15,0.25,0.15]
poro_mean=[0.3,0.3,0.1,0.15,0.1]
#absolute perm (mDarcy)
perm_min=[100,1000,1,10,1]
perm_max=[1000,10000,10,100,10]
perm_mean=[500,5000,5,50,5]
#relative permeability and capillary pressure
Slr=[0.1,0.05,0.2,0.15,0.2]
vg_lambda=[0.6,0.7,0.4,0.5,0.4]
entry_p=[1000,500,10000,5000,10000]#Pa
#Viscosities
mu_water=0.000466 #Pa.s
mu_CO2=0.000023 #Pa.s

In [126]:
#properties calculated internally

#calculating scaling parameter for Van-Genuchten capillary pressure curve
Scale_P=[None]*len(All_rock_types)
i=0
for i in range (len(All_rock_types)):
    Scale_P[i]=(entry_p[i]/((((0.995-Slr[i])/(1-Slr[i]))**(-1/vg_lambda[i]))-1)**(1-vg_lambda[i]))

#Calculating fractional flow values (to be used for upscaling rel perm under VL)
Sat_ff=[None]*14

Ff=[None]*14
krw_ff=0
krn_ff=0

#c1 and c2 are fitting paramters for ff-sat sigmoid curve
c1=[None]*len(All_rock_types) #determines curvature of the sigmoid function
c2=[None]*len(All_rock_types) #determines the centroid of the sigmoid function

j=0
i=0
for j in range(len(All_rock_types)):
    #create a dummy saturation list
    Sat_ff[0]=Slr[j]+0.01
    k=1
    for k in range (1,11):
        Sat_ff[k]=Sat_ff[k-1]+((0.95-Sat_ff[0])/10)    
    Sat_ff[11]=0.99
    Sat_ff[12]=0.995
    Sat_ff[13]=1

    #use the dummy saturation list to compute the range of fractional flow values for each homogeneous rock type
    for i in range (14):
        krw_ff=(((Sat_ff[i]-Slr[j])/((1-Slr[j])))**0.5)*(1-(1-(((Sat_ff[i]-Slr[j])/((1-Slr[j]))))**(1/vg_lambda[j]))**(vg_lambda[j]))**2
        krn_ff=((1-(((Sat_ff[i]-Slr[j])/((1-Slr[j])))))**2)*(1-(((Sat_ff[i]-Slr[j])/((1-Slr[j]))))**2)
        Ff[i]=1/(1+((mu_water/mu_CO2)*(krn_ff/krw_ff)))

    #select two points from the fraction flow-sat values for fitting to the sigmoid function
    ff1=0 #represents the highest ff values <0.5
    ff2=1 #represents the least ff value >0.5
    h=0
    for h in range(14):
        if(Ff[h]>ff1 and Ff[h]<0.5):
            ff1=Ff[h]
        if(Ff[13-h]<ff2 and Ff[13-h]>0.5):
            ff2=Ff[13-h]
    
    Sat1= Sat_ff[Ff.index(ff1)] #represents brine saturation corresponding to ff1
    Sat2= Sat_ff[Ff.index(ff2)] #represents brine saturation corresponding to ff2

    p=(np.log((1-ff1)/(ff1)))/(np.log((1-ff2)/(ff2)))
    #calculate fitting paramters of the sigmoid function for each homogeneous rock type
    c2[j]=(Sat1-(p*Sat2))/(1-p)
    c1[j]=(1/(c2[j]-Sat1))*(np.log((1-ff1)/(ff1)))


In [127]:
#Compute porosities and x,y,z abs perm of all homogeneous and composite rock types

#Dataframe for properties of all the rock types
df_rock_prop=pd.DataFrame(columns=['Rock_type','Proportion','Porosity_min','Porosity_max','Porosity_mean'
                                    'Perm_x_min','Perm_x_max','Perm_x_mean','Perm_y_min','Perm_y_max','Perm_y_mean',
                                    'Perm_z_min','Perm_z_max','Perm_z_mean'])

#Fill-in the unique rock type names from the quantified colorlith
i=0
for i in range (len(df_coarse.LITHOLOGY.unique())):
    df_rock_prop.loc[i,"Rock_type"]=df_coarse.LITHOLOGY.unique()[i]
i=0
for i in range (len(df_coarse.LITHOLOGY.unique())):
    j=0
    Carb_counter=0
    Mud_counter=0
    c=0
    
    #calculate proportions of each rock type
    for j in range(len(df_coarse)):
        if(df_rock_prop.loc[i,"Rock_type"]==df_coarse.loc[j,"LITHOLOGY"]):
            c+=1
    df_rock_prop.loc[i,"Proportion"]=(c/len(df_coarse))

    str= df_rock_prop.loc[i,"Rock_type"]
    str3=list(dict.fromkeys(str.split(" ")))
    
    #Compute min, max, mean values of porosity and abs perm in x,y,z directions if the rock type is homogeneous
    if(str3[0]=="Homogeneous"):
        df_rock_prop.loc[i,"Porosity_min"]=poro_min[All_rock_types.index(str3[1])]
        df_rock_prop.loc[i,"Porosity_max"]=poro_max[All_rock_types.index(str3[1])]
        df_rock_prop.loc[i,"Porosity_mean"]=poro_mean[All_rock_types.index(str3[1])]
        df_rock_prop.loc[i,"Perm_x_min"]=perm_min[All_rock_types.index(str3[1])]
        df_rock_prop.loc[i,"Perm_x_max"]=perm_max[All_rock_types.index(str3[1])]
        df_rock_prop.loc[i,"Perm_x_mean"]=perm_mean[All_rock_types.index(str3[1])]
        df_rock_prop.loc[i,"Perm_y_min"]=perm_min[All_rock_types.index(str3[1])]
        df_rock_prop.loc[i,"Perm_y_max"]=perm_max[All_rock_types.index(str3[1])]
        df_rock_prop.loc[i,"Perm_y_mean"]=perm_mean[All_rock_types.index(str3[1])]
        df_rock_prop.loc[i,"Perm_z_min"]=perm_min[All_rock_types.index(str3[1])]
        df_rock_prop.loc[i,"Perm_z_max"]=perm_max[All_rock_types.index(str3[1])]
        df_rock_prop.loc[i,"Perm_z_mean"]=perm_mean[All_rock_types.index(str3[1])]

    #Compute min, max, mean values of porosity and abs perm in x,y,z directions if the rock type is Composite
    if(str3[0]=='Composite'):
        k=0
        comp_poro_min=[None]*(len(str3)-2)
        comp_poro_max=[None]*(len(str3)-2)
        comp_poro_mean=[None]*(len(str3)-2)
        comp_perm_x_min=[None]*(len(str3)-2)
        comp_perm_x_max=[None]*(len(str3)-2)
        comp_perm_y_min=[None]*(len(str3)-2)
        comp_perm_y_max=[None]*(len(str3)-2)
        comp_perm_z_min=[None]*(len(str3)-2)
        comp_perm_z_max=[None]*(len(str3)-2)
        comp_perm_x_mean=[None]*(len(str3)-2)
        comp_perm_y_mean=[None]*(len(str3)-2)
        comp_perm_z_mean=[None]*(len(str3)-2)
        

        for k in range (len(str3)-2):
            a=k+2
            comp_poro_min[k]=poro_min[All_rock_types.index(str3[a])]
            comp_poro_max[k]=poro_max[All_rock_types.index(str3[a])]
            comp_poro_mean[k]=poro_mean[All_rock_types.index(str3[a])]

            l=0
            #If the composite rock has Mudstone (z-perm is harmonic while x&y perms are arithmetic avg)
            for l in range(len(Mud_Name)):
                if(Mud_Name[l]==str3[a] and Carb_counter==0):
                    Mud_counter+=1
                    v=0
                    for v in range (len(str3)-2):
                        comp_perm_x_min[v]=perm_min[All_rock_types.index(str3[v+2])]    
                        comp_perm_x_max[v]=perm_max[All_rock_types.index(str3[v+2])]
                        comp_perm_x_mean[v]=perm_mean[All_rock_types.index(str3[v+2])]
                        comp_perm_y_min[v]=perm_min[All_rock_types.index(str3[v+2])]
                        comp_perm_y_max[v]=perm_max[All_rock_types.index(str3[v+2])]
                        comp_perm_y_mean[v]=perm_mean[All_rock_types.index(str3[v+2])]
                        comp_perm_z_min[v]=perm_min[All_rock_types.index(str3[v+2])]
                        comp_perm_z_max[v]=perm_max[All_rock_types.index(str3[v+2])]
                        comp_perm_z_mean[v]=perm_mean[All_rock_types.index(str3[v+2])]
                    df_rock_prop.loc[i,"Perm_x_min"]=mean(comp_perm_x_min)
                    df_rock_prop.loc[i,"Perm_x_max"]=mean(comp_perm_x_max)
                    df_rock_prop.loc[i,"Perm_x_mean"]=mean(comp_perm_x_mean)
                    df_rock_prop.loc[i,"Perm_y_min"]=mean(comp_perm_y_min)
                    df_rock_prop.loc[i,"Perm_y_max"]=mean(comp_perm_y_max)
                    df_rock_prop.loc[i,"Perm_y_mean"]=mean(comp_perm_y_mean)
                    df_rock_prop.loc[i,"Perm_z_min"]=harmonic_mean(comp_perm_z_min)
                    df_rock_prop.loc[i,"Perm_z_max"]=harmonic_mean(comp_perm_z_max)
                    df_rock_prop.loc[i,"Perm_z_mean"]=harmonic_mean(comp_perm_z_mean)
            
            #If the composite rock has Carbonate (all perms are harmonic avg)
            l=0
            for l in range(len(Carb_Name)):
                if(Carb_Name[l]==str3[a]):
                    Carb_counter+=1
                    v=0
                    for v in range (len(str3)-2):
                        comp_perm_x_min[v]=perm_min[All_rock_types.index(str3[v+2])]    
                        comp_perm_x_max[v]=perm_max[All_rock_types.index(str3[v+2])]
                        comp_perm_x_mean[v]=perm_mean[All_rock_types.index(str3[v+2])]
                        comp_perm_y_min[v]=perm_min[All_rock_types.index(str3[v+2])]
                        comp_perm_y_max[v]=perm_max[All_rock_types.index(str3[v+2])]
                        comp_perm_y_mean[v]=perm_mean[All_rock_types.index(str3[v+2])]
                        comp_perm_z_min[v]=perm_min[All_rock_types.index(str3[v+2])]
                        comp_perm_z_max[v]=perm_max[All_rock_types.index(str3[v+2])]
                        comp_perm_z_mean[v]=perm_mean[All_rock_types.index(str3[v+2])]
                    df_rock_prop.loc[i,"Perm_x_min"]=harmonic_mean(comp_perm_x_min)
                    df_rock_prop.loc[i,"Perm_x_max"]=harmonic_mean(comp_perm_x_max)
                    df_rock_prop.loc[i,"Perm_x_mean"]=harmonic_mean(comp_perm_x_mean)
                    df_rock_prop.loc[i,"Perm_y_min"]=harmonic_mean(comp_perm_y_min)
                    df_rock_prop.loc[i,"Perm_y_max"]=harmonic_mean(comp_perm_y_max)
                    df_rock_prop.loc[i,"Perm_y_mean"]=harmonic_mean(comp_perm_y_mean)
                    df_rock_prop.loc[i,"Perm_z_min"]=harmonic_mean(comp_perm_z_min)
                    df_rock_prop.loc[i,"Perm_z_max"]=harmonic_mean(comp_perm_z_max)
                    df_rock_prop.loc[i,"Perm_z_mean"]=harmonic_mean(comp_perm_z_mean)
            
            #If the composite rock has only sanstones and neither mudstone nor carbonate (x&z perms are harmonic avg; y perms are arithmatic avg)
            l=0
            for l in range(len(Sand_Name)):
                if(Sand_Name[l]==str3[a] and Carb_counter==0 and Mud_counter==0):
                    v=0
                    for v in range (len(str3)-2):
                        comp_perm_x_min[v]=perm_min[All_rock_types.index(str3[v+2])]    
                        comp_perm_x_max[v]=perm_max[All_rock_types.index(str3[v+2])]
                        comp_perm_x_mean[v]=perm_mean[All_rock_types.index(str3[v+2])]
                        comp_perm_y_min[v]=perm_min[All_rock_types.index(str3[v+2])]
                        comp_perm_y_max[v]=perm_max[All_rock_types.index(str3[v+2])]
                        comp_perm_y_mean[v]=perm_mean[All_rock_types.index(str3[v+2])]
                        comp_perm_z_min[v]=perm_min[All_rock_types.index(str3[v+2])]
                        comp_perm_z_max[v]=perm_max[All_rock_types.index(str3[v+2])]
                        comp_perm_z_mean[v]=perm_mean[All_rock_types.index(str3[v+2])]
                    df_rock_prop.loc[i,"Perm_x_min"]=harmonic_mean(comp_perm_x_min)
                    df_rock_prop.loc[i,"Perm_x_max"]=harmonic_mean(comp_perm_x_max)
                    df_rock_prop.loc[i,"Perm_x_mean"]=harmonic_mean(comp_perm_x_mean)
                    df_rock_prop.loc[i,"Perm_y_min"]=mean(comp_perm_y_min)
                    df_rock_prop.loc[i,"Perm_y_max"]=mean(comp_perm_y_max)
                    df_rock_prop.loc[i,"Perm_y_mean"]=mean(comp_perm_y_mean)
                    df_rock_prop.loc[i,"Perm_z_min"]=harmonic_mean(comp_perm_z_min)
                    df_rock_prop.loc[i,"Perm_z_max"]=harmonic_mean(comp_perm_z_max)
                    df_rock_prop.loc[i,"Perm_z_mean"]=harmonic_mean(comp_perm_z_mean)
        df_rock_prop.loc[i,"Porosity_min"]= mean(comp_poro_min)
        df_rock_prop.loc[i,"Porosity_max"]= mean(comp_poro_max)
        df_rock_prop.loc[i,"Porosity_mean"]= mean(comp_poro_mean)


In [128]:
#Compute upscaled rel-perm and capillary pressure-saturation values
df_rock_prop_copy=pd.DataFrame(columns=['Rock_type','Proportion','Porosity_min','Porosity_max','Porosity_mean',
                                    'Perm_x_min','Perm_x_max','Perm_x_mean','Perm_y_min','Perm_y_max','Perm_y_mean',
                                    'Perm_z_min','Perm_z_max','Perm_z_mean',
                                    'Sat_cl','Pc_cl','krw_x_cl','krg_x_cl','krw_y_cl','krg_y_cl','krw_z_cl','krg_z_cl',
                                    'Sat_vl','Pc_vl','krw_x_vl','krg_x_vl','krw_y_vl','krg_y_vl','krw_z_vl','krg_z_vl'])
i=0
df_rock_prop_copy.loc[0]=0

for i in range (len(df_rock_prop)):
    str= df_rock_prop.loc[i,"Rock_type"]
    str3=list(dict.fromkeys(str.split(" ")))
    
    Sat=[None]*14
    Pc=[None]*len(Sat)
    krw_x=[None]*len(Sat)
    krg_x=[None]*len(Sat)
    krw_y=[None]*len(Sat)
    krg_y=[None]*len(Sat)
    krw_z=[None]*len(Sat)
    krg_z=[None]*len(Sat)

    #Compute saturation functions if the rock type is homogeneous 
    if(str3[0]=="Homogeneous"):
        j=0
        
        Sat[0]=Slr[0]+0.01
        k=1
        for k in range (1,11):
            Sat[k]=Sat[k-1]+((0.95-Sat[0])/10)    
        Sat[11]=0.99
        Sat[12]=0.995
        Sat[13]=1
      
        for j in range (len(Sat)):
            
            Pc[j]=Scale_P[All_rock_types.index(str3[1])]*(((Sat[j]-Slr[All_rock_types.index(str3[1])])/(1-Slr[All_rock_types.index(str3[1])]))**(-1/vg_lambda[All_rock_types.index(str3[1])])-1)**(1-vg_lambda[All_rock_types.index(str3[1])])
            krw_x[j]=(((Sat[j]-Slr[All_rock_types.index(str3[1])])/((1-Slr[All_rock_types.index(str3[1])])))**0.5)*(1-(1-(((Sat[j]-Slr[All_rock_types.index(str3[1])])/((1-Slr[All_rock_types.index(str3[1])]))))**(1/vg_lambda[All_rock_types.index(str3[1])]))**(vg_lambda[All_rock_types.index(str3[1])]))**2
            krg_x[j]=((1-(((Sat[j]-Slr[All_rock_types.index(str3[1])])/((1-Slr[All_rock_types.index(str3[1])])))))**2)*(1-(((Sat[j]-Slr[All_rock_types.index(str3[1])])/((1-Slr[All_rock_types.index(str3[1])]))))**2)
            krw_y[j]=krw_x[j]
            krw_z[j]=krw_x[j]
            krg_y[j]=krg_x[j]
            krg_z[j]=krg_x[j] 
      
        #write computed values in the output dataframe
        b=0
        for b in range(len(Sat)):
            if(b==0):
                df_rock_prop_copy=df_rock_prop_copy.append({'Rock_type': df_rock_prop.loc[i,"Rock_type"],'Proportion': df_rock_prop.loc[i,'Proportion'],
                                            'Porosity_min': df_rock_prop.loc[i,'Porosity_min'],'Porosity_max':df_rock_prop.loc[i,'Porosity_max'],
                                            'Porosity_mean':df_rock_prop.loc[i,'Porosity_mean'],
                                            'Perm_x_min':df_rock_prop.loc[i,'Perm_x_min'],'Perm_x_max':df_rock_prop.loc[i,'Perm_x_max'],
                                            'Perm_x_mean':df_rock_prop.loc[i,'Perm_x_mean'],'Perm_y_min':df_rock_prop.loc[i,'Perm_y_min'],
                                            'Perm_y_max':df_rock_prop.loc[i,'Perm_y_max'],'Perm_y_mean':df_rock_prop.loc[i,'Perm_y_mean'],
                                            'Perm_z_min':df_rock_prop.loc[i,'Perm_z_min'],'Perm_z_max':df_rock_prop.loc[i,'Perm_z_max'],
                                            'Perm_z_mean':df_rock_prop.loc[i,'Perm_z_mean'],'Sat_cl':Sat[b],'Pc_cl':Pc[b].real,'krw_x_cl':krw_x[b].real,'krg_x_cl':krg_x[b].real,
                                            'krw_y_cl':krw_y[b].real,'krg_y_cl':krg_y[b].real,'krw_z_cl':krw_z[b].real,'krg_z_cl':krg_z[b].real,
                                            'Sat_vl':Sat[b],'Pc_vl':Pc[b].real,'krw_x_vl':krw_x[b].real,'krg_x_vl':krg_x[b].real,
                                            'krw_y_vl':krw_y[b].real,'krg_y_vl':krg_y[b].real,'krw_z_vl':krw_z[b].real,'krg_z_vl':krg_z[b].real},ignore_index=True) 
            if (b>0):
                df_rock_prop_copy=df_rock_prop_copy.append({'Rock_type':"",'Proportion':"",
                                            'Porosity_min':"",'Porosity_max':"",
                                            'Porosity_mean':"",
                                            'Perm_x_min':"",'Perm_x_max':"",
                                            'Perm_x_mean':"",'Perm_y_min':"",
                                            'Perm_y_max':"",'Perm_y_mean':"",
                                            'Perm_z_min':"",'Perm_z_max':"",
                                            'Perm_z_mean':"",'Sat_cl':Sat[b],'Pc_cl':Pc[b].real,'krw_x_cl':krw_x[b].real,'krg_x_cl':krg_x[b].real,
                                            'krw_y_cl':krw_y[b].real,'krg_y_cl':krg_y[b].real,'krw_z_cl':krw_z[b].real,'krg_z_cl':krg_z[b].real,
                                            'Sat_vl':Sat[b],'Pc_vl':Pc[b].real,'krw_x_vl':krw_x[b].real,'krg_x_vl':krg_x[b].real,
                                            'krw_y_vl':krw_y[b].real,'krg_y_vl':krg_y[b].real,'krw_z_vl':krw_z[b].real,'krg_z_vl':krg_z[b].real},ignore_index=True) 
   
    
    #Compute saturation function values if the rock type is a composite
    if(str3[0]=="Composite"):
    
    #Computing under capillary limit flow regime (Pc is equal in all constitutent lithologies)
        Sat_cl=[None]*14
        Pc_cl=[None]*len(Sat)
        krw_x_cl=[None]*len(Sat)
        krg_x_cl=[None]*len(Sat)
        krw_y_cl=[None]*len(Sat)
        krg_y_cl=[None]*len(Sat)
        krw_z_cl=[None]*len(Sat)
        krg_z_cl=[None]*len(Sat)
   
        S_layer_cl=[None]*(len(str3)-2)
        K_w_eff_cl=[None]*(len(str3)-2)               
        K_g_eff_cl=[None]*(len(str3)-2) 
        
        d=-3
        for d in range (-3,11):
            Pc_cl[d+3]=10**(d) #Generate a dummy Pc list
            h=0
            Carb_counter=0
            Mud_counter=0
            
            #Calculate Sat, Krw, Krg values in each constituent lithology of the composite
            for h in range (len(str3)-2):
                #Saturation of each constituent lithology inverted from respective homogeneous Pc curves
                x=(Slr[All_rock_types.index(str3[h+2])]+(1-Slr[All_rock_types.index(str3[h+2])])*((1+((Pc_cl[d+3])/(Scale_P[All_rock_types.index(str3[h+2])]))**(1/(1-(vg_lambda[All_rock_types.index(str3[h+2])]))))**(-vg_lambda[All_rock_types.index(str3[h+2])]))).real
                
                #Ensure that the lithology-specific saturation from inverted Pc curve > Slr of the lithology
                if(x>Slr[All_rock_types.index(str3[h+2])]):
                    #porosity times saturation
                    S_layer_cl[h]=x*poro_mean[All_rock_types.index(str3[h+2])]
                    #brine rel perm of the lithology at the inverted saturation
                    y=((((x-Slr[All_rock_types.index(str3[h+2])])/((1-Slr[All_rock_types.index(str3[h+2])])))**0.5)*(1-(1-(((x-Slr[All_rock_types.index(str3[h+2])])/((1-Slr[All_rock_types.index(str3[h+2])]))))**(1/vg_lambda[All_rock_types.index(str3[h+2])]))**(vg_lambda[All_rock_types.index(str3[h+2])]))**2).real
                    #brine effective perm of the lithology at the inverted saturation
                    K_w_eff_cl[h]=y*perm_mean[All_rock_types.index(str3[h+2])]
                    #nw phase rel perm of the lithology at the inverted saturation
                    z=(((1-(((x-Slr[All_rock_types.index(str3[h+2])])/((1-Slr[All_rock_types.index(str3[h+2])])))))**2)*(1-(((x-Slr[All_rock_types.index(str3[h+2])])/((1-Slr[All_rock_types.index(str3[h+2])]))))**2)).real
                    #nw phase effective perm at the inverted saturation
                    K_g_eff_cl[h]=z*perm_mean[All_rock_types.index(str3[h+2])]
                if(x<Slr[All_rock_types.index(str3[h+2])]): #if lithology-specific saturation from inverted Pc curve < Slr of the lithology
                    #reset saturation to ~Slr
                    x=Slr[All_rock_types.index(str3[h+2])]+0.01
                    #porosity times saturation
                    S_layer_cl[h]=x*poro_mean[All_rock_types.index(str3[h+2])]
                    #brine rel perm of the lithology at the inverted saturation
                    y=((((x-Slr[All_rock_types.index(str3[h+2])])/((1-Slr[All_rock_types.index(str3[h+2])])))**0.5)*(1-(1-(((x-Slr[All_rock_types.index(str3[h+2])])/((1-Slr[All_rock_types.index(str3[h+2])]))))**(1/vg_lambda[All_rock_types.index(str3[h+2])]))**(vg_lambda[All_rock_types.index(str3[h+2])]))**2).real
                    #brine effective perm of the lithology at the inverted saturation
                    K_w_eff_cl[h]=y*perm_mean[All_rock_types.index(str3[h+2])]
                    #nw phase rel perm of the lithology at the inverted saturation
                    z=(((1-(((x-Slr[All_rock_types.index(str3[h+2])])/((1-Slr[All_rock_types.index(str3[h+2])])))))**2)*(1-(((x-Slr[All_rock_types.index(str3[h+2])])/((1-Slr[All_rock_types.index(str3[h+2])]))))**2)).real
                    #nw phase effective perm at the inverted saturation
                    K_g_eff_cl[h]=z*perm_mean[All_rock_types.index(str3[h+2])]
                if(x>1): #if lithology-specific saturation from inverted Pc curve >1
                    #reset saturation to ~1
                    x=0.99
                    #porosity times saturation
                    S_layer_cl[h]=x*poro_mean[All_rock_types.index(str3[h+2])]
                    #brine rel perm of the lithology at the inverted saturation
                    y=((((x-Slr[All_rock_types.index(str3[h+2])])/((1-Slr[All_rock_types.index(str3[h+2])])))**0.5)*(1-(1-(((x-Slr[All_rock_types.index(str3[h+2])])/((1-Slr[All_rock_types.index(str3[h+2])]))))**(1/vg_lambda[All_rock_types.index(str3[h+2])]))**(vg_lambda[All_rock_types.index(str3[h+2])]))**2).real
                    #brine effective perm of the lithology at the inverted saturation
                    K_w_eff_cl[h]=y*perm_mean[All_rock_types.index(str3[h+2])]
                    #nw phase rel perm of the lithology at the inverted saturation
                    z=(((1-(((x-Slr[All_rock_types.index(str3[h+2])])/((1-Slr[All_rock_types.index(str3[h+2])])))))**2)*(1-(((x-Slr[All_rock_types.index(str3[h+2])])/((1-Slr[All_rock_types.index(str3[h+2])]))))**2)).real
                    #nw phase effective perm at the inverted saturation
                    K_g_eff_cl[h]=z*perm_mean[All_rock_types.index(str3[h+2])]
                

            #Calculate averaged Sat, Krw, Krg of the composite from the values in each of it constituent lithologies
            k=0
            for k in range (len(str3)-2):
                a=k+2
                l=0
                #If the composite rock has Mudstone (z-perm is harmonic while x&y perms are arithmetic avg)
                for l in range(len(Mud_Name)):
                    if(Mud_Name[l]==str3[a] and Carb_counter==0):
                        Mud_counter+=1
                        v=0
                        # for v in range (len(str3)-2):
                        Sat_cl[d+3]=sum(S_layer_cl)/(df_rock_prop.loc[i,'Porosity_mean']*(len(str3)-2))
                       
                        krw_x_cl[d+3]=mean(K_w_eff_cl)/df_rock_prop.loc[i,'Perm_x_mean']
                        krw_y_cl[d+3]=mean(K_w_eff_cl)/df_rock_prop.loc[i,'Perm_y_mean']
                        krw_z_cl[d+3]=harmonic_mean(K_w_eff_cl)/df_rock_prop.loc[i,'Perm_z_mean']

                        krg_x_cl[d+3]=mean(K_g_eff_cl)/df_rock_prop.loc[i,'Perm_x_mean']
                        krg_y_cl[d+3]=mean(K_g_eff_cl)/df_rock_prop.loc[i,'Perm_y_mean']
                        krg_z_cl[d+3]=harmonic_mean(K_g_eff_cl)/df_rock_prop.loc[i,'Perm_z_mean']

                #If the composite rock has Carbonate (all perms are harmonic avg)
                l=0
                for l in range(len(Carb_Name)):
                    if(Carb_Name[l]==str3[a]):
                        Carb_counter+=1
                        v=0
                        # for v in range (len(str3)-2):
                        Sat_cl[d+3]=sum(S_layer_cl)/(df_rock_prop.loc[i,'Porosity_mean']*(len(str3)-2))
                        
                        krw_x_cl[d+3]=harmonic_mean(K_w_eff_cl)/df_rock_prop.loc[i,'Perm_x_mean']
                        krw_y_cl[d+3]=harmonic_mean(K_w_eff_cl)/df_rock_prop.loc[i,'Perm_y_mean']
                        krw_z_cl[d+3]=harmonic_mean(K_w_eff_cl)/df_rock_prop.loc[i,'Perm_z_mean']

                        krg_x_cl[d+3]=harmonic_mean(K_g_eff_cl)/df_rock_prop.loc[i,'Perm_x_mean']
                        krg_y_cl[d+3]=harmonic_mean(K_g_eff_cl)/df_rock_prop.loc[i,'Perm_y_mean']
                        krg_z_cl[d+3]=harmonic_mean(K_g_eff_cl)/df_rock_prop.loc[i,'Perm_z_mean']
                
                #If the composite rock has only sanstones and neither mudstone nor carbonate (x&z perms are harmonic avg; y perms are arithmatic avg)
                l=0
                for l in range(len(Sand_Name)):
                    if(Sand_Name[l]==str3[a] and Carb_counter==0 and Mud_counter==0):
                        v=0
                        # for v in range (len(str3)-2):
                        Sat_cl[d+3]=sum(S_layer_cl)/(df_rock_prop.loc[i,'Porosity_mean']*(len(str3)-2))
                        
                        krw_x_cl[d+3]=harmonic_mean(K_w_eff_cl)/df_rock_prop.loc[i,'Perm_x_mean']
                        krw_y_cl[d+3]=mean(K_w_eff_cl)/df_rock_prop.loc[i,'Perm_y_mean']
                        krw_z_cl[d+3]=harmonic_mean(K_w_eff_cl)/df_rock_prop.loc[i,'Perm_z_mean']

                        krg_x_cl[d+3]=harmonic_mean(K_g_eff_cl)/df_rock_prop.loc[i,'Perm_x_mean']
                        krg_y_cl[d+3]=mean(K_g_eff_cl)/df_rock_prop.loc[i,'Perm_y_mean']
                        krg_z_cl[d+3]=harmonic_mean(K_g_eff_cl)/df_rock_prop.loc[i,'Perm_z_mean']

    #Computing under viscous limit flow regime (Fractional Flow is equal in all constitutent lithologies)

        Sat_vl=[None]*14
        Pc_vl=[None]*len(Sat)
        krw_x_vl=[None]*len(Sat)
        krg_x_vl=[None]*len(Sat)
        krw_y_vl=[None]*len(Sat)
        krg_y_vl=[None]*len(Sat)
        krw_z_vl=[None]*len(Sat)
        krg_z_vl=[None]*len(Sat)

        S_layer_vl=[None]*(len(str3)-2)
        K_w_eff_vl=[None]*(len(str3)-2)               
        K_g_eff_vl=[None]*(len(str3)-2) 
        
        Pc_vl_layer=[None]*(len(str3)-2)
        d=-3
        #generate a dummy fractional flow list
        #Ff_sig=[0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.85,0.9,0.95,0.98]
        Ff_sig=[0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.81,0.82,0.83,0.84,0.9]
        for d in range (-3,11):
            h=0
            Carb_counter=0
            Mud_counter=0
            #Calculate Sat, Krw, Krg values in each constituent lithology of the composite
            for h in range (len(str3)-2):
                #Saturation of each constituent lithology inverted from respective homogeneous fractional flow curves
                x1=c2[All_rock_types.index(str3[h+2])]-(1/c1[All_rock_types.index(str3[h+2])])*(np.log((1-Ff_sig[d+3])/(Ff_sig[d+3])))
                
                #Ensure that the lithology-specific saturation from inverted Pc curve > Slr of the lithology
                if(x1>Slr[All_rock_types.index(str3[h+2])]):
                    #porosity times saturation
                    S_layer_vl[h]=x1*poro_mean[All_rock_types.index(str3[h+2])]
                    #Compute Pc from the inverted saturation
                    x2=(Scale_P[All_rock_types.index(str3[h+2])]*(((x1-Slr[All_rock_types.index(str3[h+2])])/(1-Slr[All_rock_types.index(str3[h+2])]))**(-1/vg_lambda[All_rock_types.index(str3[h+2])])-1)**(1-vg_lambda[All_rock_types.index(str3[h+2])])).real
                    #porosity times Pc
                    Pc_vl_layer[h]=x2*poro_mean[All_rock_types.index(str3[h+2])]
                    #brine rel perm of the lithology at the inverted saturation
                    y1=((((x1-Slr[All_rock_types.index(str3[h+2])])/((1-Slr[All_rock_types.index(str3[h+2])])))**0.5)*(1-(1-(((x1-Slr[All_rock_types.index(str3[h+2])])/((1-Slr[All_rock_types.index(str3[h+2])]))))**(1/vg_lambda[All_rock_types.index(str3[h+2])]))**(vg_lambda[All_rock_types.index(str3[h+2])]))**2).real
                    #brine effective perm of the lithology at the inverted saturation
                    K_w_eff_vl[h]=y1*perm_mean[All_rock_types.index(str3[h+2])]
                    #nw-phase rel perm of the lithology at the inverted saturation
                    z1=(((1-(((x1-Slr[All_rock_types.index(str3[h+2])])/((1-Slr[All_rock_types.index(str3[h+2])])))))**2)*(1-(((x1-Slr[All_rock_types.index(str3[h+2])])/((1-Slr[All_rock_types.index(str3[h+2])]))))**2)).real
                    #nw-phase effective perm of the lithology at the inverted saturation
                    K_g_eff_vl[h]=z1*perm_mean[All_rock_types.index(str3[h+2])]
                if(x1<Slr[All_rock_types.index(str3[h+2])]): #if lithology-specific saturation from inverted Pc curve < Slr of the lithology
                    #reset saturation to ~Slr
                    x1=Slr[All_rock_types.index(str3[h+2])]+0.01
                    #porosity times saturation
                    S_layer_vl[h]=x1*poro_mean[All_rock_types.index(str3[h+2])]
                    #Compute Pc from the inverted saturation
                    x2=(Scale_P[All_rock_types.index(str3[h+2])]*(((x1-Slr[All_rock_types.index(str3[h+2])])/(1-Slr[All_rock_types.index(str3[h+2])]))**(-1/vg_lambda[All_rock_types.index(str3[h+2])])-1)**(1-vg_lambda[All_rock_types.index(str3[h+2])])).real
                    #porosity times Pc
                    Pc_vl_layer[h]=x2*poro_mean[All_rock_types.index(str3[h+2])]
                    #brine rel perm of the lithology at the inverted saturation
                    y1=((((x1-Slr[All_rock_types.index(str3[h+2])])/((1-Slr[All_rock_types.index(str3[h+2])])))**0.5)*(1-(1-(((x1-Slr[All_rock_types.index(str3[h+2])])/((1-Slr[All_rock_types.index(str3[h+2])]))))**(1/vg_lambda[All_rock_types.index(str3[h+2])]))**(vg_lambda[All_rock_types.index(str3[h+2])]))**2).real
                    #brine effective perm of the lithology at the inverted saturation
                    K_w_eff_vl[h]=y1*perm_mean[All_rock_types.index(str3[h+2])]
                    #nw-phase rel perm of the lithology at the inverted saturation
                    z1=(((1-(((x1-Slr[All_rock_types.index(str3[h+2])])/((1-Slr[All_rock_types.index(str3[h+2])])))))**2)*(1-(((x1-Slr[All_rock_types.index(str3[h+2])])/((1-Slr[All_rock_types.index(str3[h+2])]))))**2)).real
                    #nw-phase effective perm of the lithology at the inverted saturation
                    K_g_eff_vl[h]=z1*perm_mean[All_rock_types.index(str3[h+2])]
                if(x1>1): #if lithology-specific saturation from inverted Pc curve < Slr of the lithology
                    #reset saturation to ~Slr
                    x1=0.99
                    #porosity times saturation
                    S_layer_vl[h]=x1*poro_mean[All_rock_types.index(str3[h+2])]
                    #Compute Pc from the inverted saturation
                    x2=(Scale_P[All_rock_types.index(str3[h+2])]*(((x1-Slr[All_rock_types.index(str3[h+2])])/(1-Slr[All_rock_types.index(str3[h+2])]))**(-1/vg_lambda[All_rock_types.index(str3[h+2])])-1)**(1-vg_lambda[All_rock_types.index(str3[h+2])])).real
                    #porosity times Pc
                    Pc_vl_layer[h]=x2*poro_mean[All_rock_types.index(str3[h+2])]
                    #brine rel perm of the lithology at the inverted saturation
                    y1=((((x1-Slr[All_rock_types.index(str3[h+2])])/((1-Slr[All_rock_types.index(str3[h+2])])))**0.5)*(1-(1-(((x1-Slr[All_rock_types.index(str3[h+2])])/((1-Slr[All_rock_types.index(str3[h+2])]))))**(1/vg_lambda[All_rock_types.index(str3[h+2])]))**(vg_lambda[All_rock_types.index(str3[h+2])]))**2).real
                    #brine effective perm of the lithology at the inverted saturation
                    K_w_eff_vl[h]=y1*perm_mean[All_rock_types.index(str3[h+2])]
                    #nw-phase rel perm of the lithology at the inverted saturation
                    z1=(((1-(((x1-Slr[All_rock_types.index(str3[h+2])])/((1-Slr[All_rock_types.index(str3[h+2])])))))**2)*(1-(((x1-Slr[All_rock_types.index(str3[h+2])])/((1-Slr[All_rock_types.index(str3[h+2])]))))**2)).real
                    #nw-phase effective perm of the lithology at the inverted saturation
                    K_g_eff_vl[h]=z1*perm_mean[All_rock_types.index(str3[h+2])]
            
            
            #Calculate averaged Sat, Pc, Krw, Krg of the composite from the values in each of its constituent lithologies
            k=0
            for k in range (len(str3)-2):
                a=k+2
                l=0
                #If the composite rock has Mudstone (z-perm is harmonic while x&y perms are arithmetic avg)
                for l in range(len(Mud_Name)):
                    if(Mud_Name[l]==str3[a] and Carb_counter==0):
                        Mud_counter+=1
                        v=0
                        # for v in range (len(str3)-2):
                        Sat_vl[d+3]=sum(S_layer_vl)/(df_rock_prop.loc[i,'Porosity_mean']*(len(str3)-2))
                        Pc_vl[d+3]=sum(Pc_vl_layer)/(df_rock_prop.loc[i,'Porosity_mean']*(len(str3)-2))

                        krw_x_vl[d+3]=mean(K_w_eff_vl)/df_rock_prop.loc[i,'Perm_x_mean']
                        krw_y_vl[d+3]=mean(K_w_eff_vl)/df_rock_prop.loc[i,'Perm_y_mean']
                        krw_z_vl[d+3]=harmonic_mean(K_w_eff_vl)/df_rock_prop.loc[i,'Perm_z_mean']

                        krg_x_vl[d+3]=mean(K_g_eff_vl)/df_rock_prop.loc[i,'Perm_x_mean']
                        krg_y_vl[d+3]=mean(K_g_eff_vl)/df_rock_prop.loc[i,'Perm_y_mean']
                        krg_z_vl[d+3]=harmonic_mean(K_g_eff_vl)/df_rock_prop.loc[i,'Perm_z_mean']
                
                #If the composite rock has Carbonate (all perms are harmonic avg)
                l=0
                for l in range(len(Carb_Name)):
                    if(Carb_Name[l]==str3[a]):
                        Carb_counter+=1
                        v=0
                        # for v in range (len(str3)-2):
                        Sat_vl[d+3]=sum(S_layer_vl)/(df_rock_prop.loc[i,'Porosity_mean']*(len(str3)-2))
                        Pc_vl[d+3]=sum(Pc_vl_layer)/(df_rock_prop.loc[i,'Porosity_mean']*(len(str3)-2))
                        
                        krw_x_vl[d+3]=harmonic_mean(K_w_eff_vl)/df_rock_prop.loc[i,'Perm_x_mean']
                        krw_y_vl[d+3]=harmonic_mean(K_w_eff_vl)/df_rock_prop.loc[i,'Perm_y_mean']
                        krw_z_vl[d+3]=harmonic_mean(K_w_eff_vl)/df_rock_prop.loc[i,'Perm_z_mean']

                        krg_x_vl[d+3]=harmonic_mean(K_g_eff_vl)/df_rock_prop.loc[i,'Perm_x_mean']
                        krg_y_vl[d+3]=harmonic_mean(K_g_eff_vl)/df_rock_prop.loc[i,'Perm_y_mean']
                        krg_z_vl[d+3]=harmonic_mean(K_g_eff_vl)/df_rock_prop.loc[i,'Perm_z_mean']
                 
                #If the composite rock has only sanstones and neither mudstone nor carbonate (x&z perms are harmonic avg; y perms are arithmatic avg)
                l=0
                for l in range(len(Sand_Name)):
                    if(Sand_Name[l]==str3[a] and Carb_counter==0 and Mud_counter==0):
                        v=0
                        # for v in range (len(str3)-2):
                        Sat_vl[d+3]=sum(S_layer_vl)/(df_rock_prop.loc[i,'Porosity_mean']*(len(str3)-2))
                        Pc_vl[d+3]=sum(Pc_vl_layer)/(df_rock_prop.loc[i,'Porosity_mean']*(len(str3)-2))
                        krw_x_vl[d+3]=harmonic_mean(K_w_eff_vl)/df_rock_prop.loc[i,'Perm_x_mean']
                        krw_y_vl[d+3]=mean(K_w_eff_vl)/df_rock_prop.loc[i,'Perm_y_mean']
                        krw_z_vl[d+3]=harmonic_mean(K_w_eff_vl)/df_rock_prop.loc[i,'Perm_z_mean']

                        krg_x_vl[d+3]=harmonic_mean(K_g_eff_vl)/df_rock_prop.loc[i,'Perm_x_mean']
                        krg_y_vl[d+3]=mean(K_g_eff_vl)/df_rock_prop.loc[i,'Perm_y_mean']
                        krg_z_vl[d+3]=harmonic_mean(K_g_eff_vl)/df_rock_prop.loc[i,'Perm_z_mean']



        # print the values to the dataframe
        b=0
        for b in range(len(Sat)):
            if (b==0):
                df_rock_prop_copy=df_rock_prop_copy.append({'Rock_type': df_rock_prop.loc[i,"Rock_type"],'Proportion': df_rock_prop.loc[i,'Proportion'],
                                            'Porosity_min': df_rock_prop.loc[i,'Porosity_min'],'Porosity_max':df_rock_prop.loc[i,'Porosity_max'],
                                            'Porosity_mean':df_rock_prop.loc[i,'Porosity_mean'],
                                            'Perm_x_min':df_rock_prop.loc[i,'Perm_x_min'],'Perm_x_max':df_rock_prop.loc[i,'Perm_x_max'],
                                            'Perm_x_mean':df_rock_prop.loc[i,'Perm_x_mean'],'Perm_y_min':df_rock_prop.loc[i,'Perm_y_min'],
                                            'Perm_y_max':df_rock_prop.loc[i,'Perm_y_max'],'Perm_y_mean':df_rock_prop.loc[i,'Perm_y_mean'],
                                            'Perm_z_min':df_rock_prop.loc[i,'Perm_z_min'],'Perm_z_max':df_rock_prop.loc[i,'Perm_z_max'],
                                            'Perm_z_mean':df_rock_prop.loc[i,'Perm_z_mean'],'Sat_cl':Sat_cl[b],'Pc_cl':Pc_cl[b].real,'krw_x_cl':krw_x_cl[b].real,'krg_x_cl':krg_x_cl[b].real,
                                            'krw_y_cl':krw_y_cl[b].real,'krg_y_cl':krg_y_cl[b].real,'krw_z_cl':krw_z_cl[b].real,'krg_z_cl':krg_z_cl[b].real,
                                            'Sat_vl':Sat_vl[b],'Pc_vl':Pc_vl[b].real,'krw_x_vl':krw_x_vl[b].real,'krg_x_vl':krg_x_vl[b].real,
                                            'krw_y_vl':krw_y_vl[b].real,'krg_y_vl':krg_y_vl[b].real,'krw_z_vl':krw_z_vl[b].real,'krg_z_vl':krg_z_vl[b].real},ignore_index=True) 
            if (b>0):
                df_rock_prop_copy=df_rock_prop_copy.append({'Rock_type':"",'Proportion':"",
                                            'Porosity_min':"",'Porosity_max':"",
                                            'Porosity_mean':"",
                                            'Perm_x_min':"",'Perm_x_max':"",
                                            'Perm_x_mean':"",'Perm_y_min':"",
                                            'Perm_y_max':"",'Perm_y_mean':"",
                                            'Perm_z_min':"",'Perm_z_max':"",
                                            'Perm_z_mean':"",'Sat_cl':Sat_cl[b],'Pc_cl':Pc_cl[b].real,'krw_x_cl':krw_x_cl[b].real,'krg_x_cl':krg_x_cl[b].real,
                                            'krw_y_cl':krw_y_cl[b].real,'krg_y_cl':krg_y_cl[b].real,'krw_z_cl':krw_z_cl[b].real,'krg_z_cl':krg_z_cl[b].real,
                                            'Sat_vl':Sat_vl[b],'Pc_vl':Pc_vl[b].real,'krw_x_vl':krw_x_vl[b].real,'krg_x_vl':krg_x_vl[b].real,
                                            'krw_y_vl':krw_y_vl[b].real,'krg_y_vl':krg_y_vl[b].real,'krw_z_vl':krw_z_vl[b].real,'krg_z_vl':krg_z_vl[b].real},ignore_index=True) 

In [129]:
#Output the rock property values
df_rock_prop_copy.loc[df_rock_prop_copy['Rock_type'].duplicated(), 'Rock_type'] = ''
df_rock_prop_copy.drop(0, axis= 0, inplace=True)
df_rock_prop_copy.to_csv('Output_Rock_Properties.csv', index=False)